# Reading the data

In this notebook we will read the train dataset, preprocessed it and export it because this work is really time and memory consuming.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_folder = "../input/"

## Assets

Let's first open the asset_details.csv file provided in the dataset.

In [3]:
df_asset = pd.read_csv(data_folder + "asset_details.csv", index_col=0)
df_asset.sort_values(by=['Weight'], ascending=False)

,Weight,Asset_Name
Asset_ID,,
1,6.779922,Bitcoin
6,5.894403,Ethereum
3,4.406719,Cardano
0,4.304065,Binance Coin
4,3.555348,Dogecoin
2,2.397895,Bitcoin Cash
9,2.397895,Litecoin
7,2.079442,Ethereum Classic
12,2.079442,Stellar


 * __Assert_ID__: the id of the asset used in the others datasets
 * __Weight__: represent the relative importance of the asset in the evaluation metric.
 * __Asset_Name__: the full name of the coin

## Train data

Now let's open the train.csv file, the biggest provided in the dataset.

### Presentation

In [4]:
df = pd.read_csv(data_folder + "train.csv")
df.head(100)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.580000,2399.5000,2357.1400,2374.590000,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.530000,8.5300,8.5300,8.530000,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.8000,13666.1100,13850.176000,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.6596,7.6567,7.657600,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.9200,25.8740,25.877000,121.087310,25.891363,-0.008264
...,...,...,...,...,...,...,...,...,...,...
95,1514765520,11,8.0,327.670000,327.6800,326.7700,326.770000,17.901633,326.990493,NaN
96,1514765580,2,38.0,2366.833333,2398.2000,2340.6000,2365.376667,8.981063,2364.424826,-0.000176
97,1514765580,0,24.0,8.399100,8.4559,8.3573,8.420000,604.390000,8.413630,-0.007573
98,1514765580,1,257.0,13690.790000,13900.0000,13473.4700,13696.638000,21.794143,13691.957069,-0.003605



 * __timestamp__: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
 * __Asset_ID__: The asset ID corresponding to one of the crytocurrencies (e.g. Asset_ID = 1 for Bitcoin). The mapping from Asset_ID to crypto asset is contained in asset_details.csv.
 * __Count__: Total number of trades in the time interval (last minute).
 * __Open__: Opening price of the time interval (in USD).
 * __High__: Highest price reached during time interval (in USD).
 * __Low__: Lowest price reached during time interval (in USD).
 * __Close__: Closing price of the time interval (in USD).
 * __Volume__: Quantity of asset bought or sold, displayed in base currency USD.
 * __VWAP__: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
 * __Target__: Residual log-returns for the asset over a 15 minute horizon.


### Filling the gaps

#### Null values

Let's start to check where are the null values in the columns

In [5]:
print(df.isna().sum())

timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
VWAP              9
Target       750338
dtype: int64


###### The VWAP column

In [6]:
null_index = df['VWAP'].isna()
df[null_index]

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
15004269,1592173560,10,2.0,501.00,501.00,501.00,501.00,0.0,NaN,NaN
15004283,1592173620,10,4.0,501.00,501.00,501.00,501.00,0.0,NaN,NaN
15059232,1592426160,10,2.0,529.77,529.77,529.77,529.77,0.0,NaN,0.007356
15143187,1592823720,10,2.0,503.60,503.60,503.60,503.60,0.0,NaN,-0.005672
15183088,1593008940,10,2.0,484.16,484.16,484.16,484.16,0.0,NaN,-0.004281
15184216,1593013920,10,2.0,480.00,480.00,480.00,480.00,0.0,NaN,NaN
15184243,1593014040,10,2.0,480.00,480.00,480.00,480.00,0.0,NaN,NaN
15184309,1593014340,10,6.0,479.07,479.07,479.07,479.07,0.0,NaN,NaN
15184778,1593016440,10,4.0,478.00,478.00,475.00,478.00,0.0,NaN,NaN


The problem with the VWAP column is that the value is NaN when the volume exchanged is equal to zero. Then the values of Open, High, Low and Close column are equal. To fix that we can simply set a new value as the average value of the asset over that period.

In [7]:
df.loc[null_index, 'VWAP'] = (df[null_index]['Open'] +  df[null_index]['Close']) / 2

##### The Target column

In [8]:
# this line gives the indexes of bitcoin with a null target
# df[(df['Asset_ID'] == 1) & (df['Target'].isna())]

# lets use one of the timestamp and see what is around
dfout = df[  (df['Asset_ID'] == 1)
           & (df['timestamp'] <= 1571201700 + 60 * 20)
           & (df['timestamp'] >= 1571201700 - 60 * 20)]
dfout[5:30]

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
10658801,1571200800,1,6.0,8171.633333,8172.810000,8170.39,8171.610000,0.789268,8171.543437,-0.000398
10658805,1571200860,1,16.0,8172.400000,8174.000000,8172.20,8172.920000,0.626580,8172.826200,-0.000876
10658811,1571200920,1,7.0,8174.982500,8195.400000,8166.74,8175.837500,0.058808,8175.271439,-0.000333
10658818,1571200980,1,4.0,8168.443333,8169.100000,8167.17,8168.443333,0.044049,8168.443333,-0.000874
10658826,1571201040,1,16.0,8174.410595,8195.402973,8165.05,8173.572595,2.801392,8173.984860,-0.000220
10658833,1571201100,1,27.0,8165.950000,8166.120000,8163.69,8164.760000,0.542897,8164.951366,0.000306
10658840,1571201160,1,22.0,8165.666667,8167.610000,8163.34,8165.916667,1.390160,8166.130653,-0.001039
10658847,1571201220,1,20.0,8176.796667,8195.400000,8166.61,8177.453333,0.734043,8177.274471,0.000199
10658853,1571201280,1,24.0,8166.544000,8168.840000,8164.06,8166.620000,0.951659,8166.545267,-0.000302
10658857,1571201340,1,16.0,8171.394000,8195.200000,8162.96,8171.392000,1.042791,8171.496016,0.000099


Here we don't really get why the target is NaN.

In [9]:
n = 15
# values of the moment
a = dfout['VWAP'][:-n].to_numpy()
# values in 15 minutes
b = dfout['Low'][n:].to_numpy()

# printing our estimation next to the true value
dfout = dfout[:-n]
dfout['nTarget'] = np.log(b / a)
dfout

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,nTarget
10658759,1571200500,1,16.0,8181.173904,8194.961713,8173.61,8182.100571,0.906794,8181.329969,0.000333,-0.002290
10658769,1571200560,1,16.0,8175.745000,8176.550000,8174.57,8175.480000,4.056520,8175.294511,-0.000181,-0.001515
10658775,1571200620,1,13.0,8171.933333,8175.350000,8168.58,8171.983333,0.828670,8172.262017,-0.000688,-0.000823
10658784,1571200680,1,12.0,8173.310000,8175.140000,8171.68,8173.796667,0.312988,8173.354084,-0.000247,-0.001035
10658793,1571200740,1,5.0,8171.805000,8172.690000,8170.92,8171.170000,0.106450,8171.723802,-0.000509,-0.000576
10658801,1571200800,1,6.0,8171.633333,8172.810000,8170.39,8171.610000,0.789268,8171.543437,-0.000398,-0.000933
10658805,1571200860,1,16.0,8172.400000,8174.000000,8172.20,8172.920000,0.626580,8172.826200,-0.000876,-0.000739
10658811,1571200920,1,7.0,8174.982500,8195.400000,8166.74,8175.837500,0.058808,8175.271439,-0.000333,-0.001256
10658818,1571200980,1,4.0,8168.443333,8169.100000,8167.17,8168.443333,0.044049,8168.443333,-0.000874,-0.000532
10658826,1571201040,1,16.0,8174.410595,8195.402973,8165.05,8173.572595,2.801392,8173.984860,-0.000220,-0.001357


The formula used here is clearly not the good one. As explained in [this notebook](https://www.kaggle.com/alexfir/recreating-target/notebook),  recreating the target is much more difficult than expected. It is not only the variation of the asset price on 15 minutes, but implies much more variables.

In order to start a model before ruling that strong problem, we'll fill the Nan values with zeros.

In [10]:
df['Target'] = df['Target'].fillna(0)
df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264
...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,0.000000
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,0.000000
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,0.000000
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,0.000000


#### Filling the missing indexes

There are a lot of missing entries, sometimes it is one, sometimes two in a row, and it goes up to four. To fix that the data is extracted by assets. The timestamp is setted as the index and we are reindexing the data from the beggining to the end of the data range. The method used here is the pad method that tropagates the last valid observation forward to next valid.

In [11]:
start_size = len(df)

df_ = None
for i in range(len(df_asset)):
    asset = df_asset['Asset_Name'][i]
    
    # temporary index key are set to timestamp to fill gaps
    df_tmp = df[df['Asset_ID'] == i].set_index("timestamp")
    
    filling_rate = 100 * ((df_tmp.index[1:] - df_tmp.index[:-1]) != 60).sum() / df_tmp.size
    df_tmp = df_tmp.reindex(range(df_tmp.index[0], df_tmp.index[-1] + 60, 60), method='pad')
    print(f"{asset}: {df_tmp.size} values, filling rate of {round(filling_rate, 3)}%")
    
    
    # checking if all indexes are continuous
    if not ((df_tmp.index[1:] - df_tmp.index[:-1]) == 60).all():
        raise Exception("Indexes are not continuous")
    
    # checking if all the values are set
    if not df_tmp.drop(columns=['Target']).isna().sum().sum() == 0:
        print(df_tmp.drop(columns=['Target']).isna().sum())
        print(df.drop(columns=['Target']).isna().sum())
        raise Exception("Somes values are not set")
    
    # setting the index key back
    df_tmp = df_tmp.reset_index()
        
    # adding in a new dataset
    if df_ is None:
        df_ = df_tmp
    else:
        df_ = df.append(df_tmp, ignore_index=True)
    
    del df_tmp

end_size = len(df_)

print(f'df size before: {start_size} entries')
print(f'df size after : {end_size} entries')
print('Increased by {:.1f}%'.format(100 * (end_size - start_size) / start_size)) 

Binance Coin: 17612640 values, filling rate of 0.038%
Bitcoin: 17612640 values, filling rate of 0.001%
Bitcoin Cash: 17612640 values, filling rate of 0.014%
Cardano: 16233930 values, filling rate of 0.058%
Dogecoin: 11565423 values, filling rate of 0.736%
EOS.IO: 17612640 values, filling rate of 0.007%
Ethereum: 17612640 values, filling rate of 0.001%
Ethereum Classic: 17612640 values, filling rate of 0.027%
IOTA: 15949386 values, filling rate of 0.752%
Litecoin: 17612640 values, filling rate of 0.002%
Maker: 15932520 values, filling rate of 0.439%
Monero: 17612640 values, filling rate of 0.861%
Stellar: 17003592 values, filling rate of 0.236%
TRON: 17134416 values, filling rate of 0.072%
df size before: 24236806 entries
df size after : 26140630 entries
Increased by 7.9%


Once it is done, df is deleted to free the RAM.

In [12]:
df = df_
df_ = df_[1:1]
del df_

### Downsizing the dataset

In [13]:
df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26140630 entries, 0 to 26140629
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   Asset_ID   int64  
 2   Count      float64
 3   Open       float64
 4   High       float64
 5   Low        float64
 6   Close      float64
 7   Volume     float64
 8   VWAP       float64
 9   Target     float64
dtypes: float64(8), int64(2)
memory usage: 1.9 GB


In [14]:
start_mem = df.memory_usage().sum() / 1024**2
df['Count'] = pd.to_numeric(df['Count'], downcast='integer')
for column in df:
    if df[column].dtype == 'float64':
        df[column] = pd.to_numeric(df[column], downcast='float')
    if df[column].dtype == 'int64':
        df[column] = pd.to_numeric(df[column], downcast='integer')
end_mem = df.memory_usage().sum() / 1024**2

In [15]:
df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26140630 entries, 0 to 26140629
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int32  
 1   Asset_ID   int8   
 2   Count      int32  
 3   Open       float32
 4   High       float32
 5   Low        float32
 6   Close      float32
 7   Volume     float32
 8   VWAP       float32
 9   Target     float32
dtypes: float32(7), int32(2), int8(1)
memory usage: 922.4 MB


In [16]:
print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

Decreased by 53.7%


## Exporting the values

In [19]:
df.to_csv(data_folder + "train_processed.csv")